In [1]:
import pandas as pd
from ipywidgets import interactive
import ipywidgets as widgets
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter, HourLocator

d = "Data//"
flis = ["LT1", "LT2", "LT3", "LT4", "LT5", "LT6", "LT7", "LT8"]
llist = ['Load', 'Unit 1', 'Unit 2', 'Unit 3', 'Unit 4']
load = pd.read_excel(d+"Load.xlsx")
load['Time'] = pd.to_datetime(load['Time'].astype(str))
weather = pd.read_excel(d+"Weather.xlsx")
weather['Time'] = pd.to_datetime(weather['Time'].astype(str))
wdir = ["N","NNE","NE","ENE","E","ESE","SE","SSE","S","SSW","SW","WSW","W","WNW","NW","NNW","---"]
lt = [[] for i in range(8)]
ltdc = [[] for i in range(8)]
dca, dcb = '07:00', '04:00'
lcrit = 1

for i, j in enumerate(flis):
    lt[i] = pd.read_excel(d+flis[i]+".xlsx", sheet_name="L90(LN4)")
    lt[i]['Start Time'] = pd.to_datetime(lt[i]['Start Time'].astype(str))
    lt[i] = pd.merge_asof(lt[i],load,left_on="Start Time", right_on="Time")
    lt[i] = lt[i].drop(labels=['Time'],axis=1)
    lt[i] = pd.merge_asof(lt[i],weather,left_on="Start Time", right_on="Time")
    lt[i] = lt[i].drop(labels=['Time'],axis=1)
    lt[i] = lt[i].set_index('Start Time')
    lt[i] = lt[i].tail(lt[i].shape[0] - 24) # removing first 6 hours from tail
    lt[i] = lt[i].head(lt[i].shape[0] - 24) # removing first 6 hours from head

%matplotlib widget
fig, ax1 = plt.subplots(1, 1, figsize=(8,4))
def plothist(n, d, c, l, w):
    ax1.clear()
    n = int(n-1)
    bins = range(15,70)
    if d:
        t, st, et  = 'Night Time','23:00', '07:00'
    else:
        t, st, et = 'Day Time', '07:00', '23:00'
    ltall = lt[n].between_time(st, et) #filtering results for day and night
    ltall = ltall[ltall["Rain"]<0.2]
    ltall = ltall[ltall["Speed"]<=5]
    ltall = ltall[ltall["Dir"].isin(w)]
    ltdcall = ltall.between_time(dca, dcb) #filtering for dawn chorus
    lton, ltoff = ltall[ltall[l]>=lcrit], ltall[ltall[l]<lcrit] #filtering results with and without drax power
    ltdcon,ltdcoff = ltdcall[ltdcall[l]>=lcrit],ltdcall[ltdcall[l]<lcrit] #filtering dawn chorus results with and without drax power
    if c:
        t2 = ' without dawn chorus'
        ax1.hist(ltdcoff['Main'],bins=bins,density=1,alpha=0.7)
        ax1.hist(ltdcon['Main'],bins=bins,density=1,alpha=0.7)
    else:
        t2 = ''
        ax1.hist(ltoff['Main'],bins=bins,density=1,alpha=0.7)
        ax1.hist(lton['Main'],bins=bins,density=1,alpha=0.7)
    plt.title(t+' at LT'+str(n+1)+t2)
    ax1.legend(['Load <'+str(lcrit), 'Load >='+str(lcrit)])
    fig.show()
box_layout = widgets.Layout(display='inline-flex',flex_flow='column',align_items='center')
ss = widgets.Select(options=llist,description='Load value: ',disabled=False)
aw = widgets.Select(options=list(range(1,9)),description='LT: ',disabled=False)
bw = widgets.ToggleButton(value=False,description='Day / Night',disabled=False)
cw = widgets.Checkbox(value=False,description='Filter dawn Chorus',disabled=False)
wd = widgets.SelectMultiple(options=wdir,value=[wdir[0]],description='wind dir', disabled=False)
ui = widgets.VBox([widgets.HBox([aw, ss, wd]),widgets.HBox([bw, cw])],layout=box_layout)
out = widgets.interactive_output(plothist, {'n': aw, 'd': bw, 'c': cw, 'l':ss,'w':wd})
display(ui,out)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Output()